In [1]:
import zipfile

# Unzip training set
local_zip = './happy-or-sad.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./happy-or-sad')

zip_ref.close()

In [3]:
import os

# Directory with training happy pictures
train_happy_dir = os.path.join('./happy-or-sad/happy')

# Directory with training sad pictures
train_sad_dir = os.path.join('./happy-or-sad/sad')

print(f'total training happy images: {len(os.listdir(train_happy_dir))}')
print(f'total training sad images: {len(os.listdir(train_sad_dir))}')

total training happy images: 40
total training sad images: 40


In [4]:

train_happy_names = os.listdir(train_happy_dir)
print(f'TRAIN SET HAPPY: {train_happy_names[:10]}')

train_sad_names = os.listdir(train_sad_dir)
print(f'TRAIN SET SAD: {train_sad_names[:10]}')

TRAIN SET HAPPY: ['happy1-19.png', 'happy1-18.png', 'happy1-08.png', 'happy1-09.png', 'happy2-17.png', 'happy2-03.png', 'happy2-02.png', 'happy2-16.png', 'happy2-00.png', 'happy2-14.png']
TRAIN SET SAD: ['sad1-09.png', 'sad1-08.png', 'sad1-18.png', 'sad1-19.png', 'sad2-10.png', 'sad2-04.png', 'sad2-05.png', 'sad2-11.png', 'sad2-07.png', 'sad2-13.png']


In [5]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

# Index for iterating over images
pic_index = 0

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_happy_pix = [os.path.join(train_happy_dir, fname) 
                for fname in train_happy_names[pic_index-8:pic_index]]
next_sad_pix = [os.path.join(train_sad_dir, fname) 
                for fname in train_sad_names[pic_index-8:pic_index]]

for i, img_path in enumerate(next_happy_pix+next_sad_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [42]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np

Building a model with 5 convolutions and pooling layers and then flatten the result to feed it to the dense layers. Since it is a binary classification we have 1 neuron in output layer with sigmoid activation layer

In [43]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 74, 74, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_23 (Conv2D)          (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_23 (MaxPooli  (None, 36, 36, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_24 (Conv2D)          (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 17, 17, 64)       

In [44]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

Creating training dataset using ImageDataGenerator. Loads training images of horse and humans from the directory and resizes them to 300x300 pixels.

In [45]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        './happy-or-sad/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 300x300
        batch_size=10,
        # Since you use binary_crossentropy loss, you need binary labels
        class_mode='binary')

Found 80 images belonging to 2 classes.


Creating a callback for accuracy >= 0.999

In [46]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.999:
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True    

callbacks = myCallback()

Training the dataset for 15 epochs. Since we have 1024 images with batch size 128, steps per epoch is 8

In [47]:
history = model.fit(
      train_generator,
      epochs=20,
      verbose=1,
      callbacks=[callbacks])

Epoch 1/20
8/8 [==============================] - 1s 76ms/step - loss: 0.7333 - accuracy: 0.5875
Epoch 2/20
8/8 [==============================] - 1s 79ms/step - loss: 0.5465 - accuracy: 0.7500
Epoch 3/20
8/8 [==============================] - 1s 83ms/step - loss: 0.2180 - accuracy: 0.9375
Epoch 4/20
8/8 [==============================] - 1s 96ms/step - loss: 0.1282 - accuracy: 0.9250
Epoch 5/20
8/8 [==============================] - 1s 83ms/step - loss: 0.0523 - accuracy: 0.9875
Epoch 6/20
8/8 [==============================] - ETA: 0s - loss: 0.0388 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 1s 88ms/step - loss: 0.0388 - accuracy: 1.0000
